In [19]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [18]:
IMG_SIZE = 150
categories = ["NORMAL", "PNEUMONIA"]

def create_training_data(DATADIR="../chest_xray/train"):
    training_data = []
    for category in categories:

        path = os.path.join(DATADIR,category)
        class_num = categories.index(category) 

        for img in tqdm(os.listdir(path)):
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE)) 
                if class_num == 0:
                    array = [1, 0]
                else:
                    array = [0, 1]
                training_data.append([new_array, array])
            except Exception as e: 
                pass

    return training_data

training_data = create_training_data("../chest_xray/train")

100%|██████████| 3875/3875 [00:15<00:00, 255.45it/s]


In [20]:
test_data = create_training_data("../chest_xray/test")

100%|██████████| 390/390 [00:01<00:00, 289.65it/s]


In [21]:
val_data = create_training_data("../chest_xray/val")

100%|██████████| 8/8 [00:00<00:00, 249.15it/s]


In [22]:
random.shuffle(training_data)
random.shuffle(test_data)
random.shuffle(val_data)

In [23]:
x_train, x_test, x_val = [], [], []
y_train, y_test, y_val = [], [], []


In [24]:
for features,label in training_data:
    x_train.append(features)
    y_train.append(label)
    
for features,label in test_data:
    x_test.append(features)
    y_test.append(label)

for features,label in val_data:
    x_val.append(features)
    y_val.append(label)


x_train = np.array(x_train).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_train = np.array(y_train)

x_test = np.array(x_test).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_test = np.array(y_test)

x_val = np.array(x_val).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_val = np.array(y_val)

In [ ]:
model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu', padding="same", input_shape=x_train.shape[1:]))
model.add(Conv2D(16, (3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu', padding="same", input_shape=x_train.shape[1:]))
model.add(Conv2D(32, (3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu', padding="same"))
model.add(Conv2D(64, (3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(96, (3, 3), dilation_rate=(2, 2), activation='relu', padding="same"))
model.add(Conv2D(96, (3, 3), padding="valid", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), dilation_rate=(2, 2), activation='relu', padding="same"))
model.add(Conv2D(128, (3, 3), padding="valid", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2 , activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=3, validation_data=(x_val, y_val))

Train on 5216 samples, validate on 16 samples
Epoch 1/3
1568/5216 [========>.....................] - ETA: 1:13 - loss: 0.7092 - accuracy: 0.7290